# Required Libraries

In [1]:
!pip install chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 26.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 84.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.1/103.1 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 93.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.0/120.0 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.6/201.6 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 11.1 MB/s et

In [2]:
import os
import json
import torch
import chromadb
import requests
import numpy as np
from PIL import Image
from io import BytesIO
from chromadb.config import Settings
from transformers import CLIPVisionModel, RobertaModel, AutoTokenizer, CLIPFeatureExtractor

# Download Data

In [3]:
!git clone https://github.com/FaSha20/Natural-Language-Processing-Projects

Cloning into 'Natural-Language-Processing-Projects'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 80 (delta 21), reused 41 (delta 9), pack-reused 0 (from 0)
Receiving objects: 100% (80/80), 8.09 MiB | 9.73 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [4]:
with open("/content/Natural-Language-Processing-Projects/MultimodalRAG/scientists_data/scintists_data_with_text_chunks.json", "r", encoding="utf-8") as f:
    data = json.load(f)

print(len(data), "records loaded")

300 records loaded


# Build Embeddings and DATABASE

## Load Pre-trained Encoder Model

In [5]:
vision_encoder = CLIPVisionModel.from_pretrained('SajjadAyoubi/clip-fa-vision')
preprocessor = CLIPFeatureExtractor.from_pretrained('SajjadAyoubi/clip-fa-vision')
text_encoder = RobertaModel.from_pretrained('SajjadAyoubi/clip-fa-text')
tokenizer = AutoTokenizer.from_pretrained('SajjadAyoubi/clip-fa-text')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/350M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/clip/feature_extraction_clip.py:30: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/354 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

## Encoding Functions

In [ ]:
# ---- Text Encoder ----
def embed_text(text: str):
    text_embedding = text_encoder(**tokenizer(text, return_tensors='pt')).pooler_output
    return text_embedding[0]

# ---- Image Encoder ----
def embed_image(url: str):
    try:
        headers = {"User-Agent": "ImageDatasetScript/1.0 (email@gmail.com)"}
        response = requests.get(url, headers=headers, timeout=10)
        image = Image.open(BytesIO(response.content)).convert("RGB")
        image_embedding = vision_encoder(**preprocessor(image, return_tensors='pt')).pooler_output
        return image_embedding[0]
    except Exception as e:
        return None

## Build the DATABASE

In [ ]:
PERSIST_DIR = "/content/chroma_scientists_db"
os.makedirs(PERSIST_DIR, exist_ok=True)

client = chromadb.PersistentClient(path=PERSIST_DIR, settings=Settings(allow_reset=True))

text_db  = client.get_or_create_collection(name="scientists_text",  metadata={"hnsw:space": "cosine"})
image_db = client.get_or_create_collection(name="scientists_image", metadata={"hnsw:space": "cosine"})

In [10]:
def upsert_pair(idx, name, text, image_url, t_emb, i_emb):
    meta = {
        "pair_id": idx,
        "name": name,
        "text": text,
        "image_url": image_url if image_url is not None else "",
    }

    if t_emb is not None:
        text_db.add(
            ids=[f"text-{idx}"],
            embeddings=[t_emb.tolist()],
            metadatas=[meta],
        )

    if i_emb is not None:
        image_db.add(
            ids=[f"img-{idx}"],
            embeddings=[i_emb.tolist()],
            metadatas=[meta],
        )

In [11]:
for idx, person in enumerate(data):
    name = person.get("name")

    text = person.get("text_chunk", "").replace("\u200c", "")
    t_emb = embed_text(text)

    image_urls = list(person.get("image", {}).values())
    img_url = None
    for url in image_urls:
        if url:
            i_emb = embed_image(url)
            if i_emb is not None:
              img_url = url
              break

    upsert_pair(idx, name, text, img_url, t_emb, i_emb)

## Retrieval Functions

In [12]:
def search_by_text(query: str, k=3):
    q_emb = embed_text(query)
    res = text_db.query(query_embeddings=[q_emb.tolist()], n_results=k, include=["metadatas", "distances"])
    return res

def search_by_image(image_url: str, k=5):
    q_emb = embed_image(image_url)
    if q_emb is None:
        return None
    res = image_db.query(query_embeddings=[q_emb.tolist()], n_results=k, include=["metadatas", "distances"])
    return res

## Test an example

In [13]:
res = search_by_text("چه کسی شرح کتاب مابعدالطبیعه ارسطو را نوشت؟", k=7)
for meta, dist in zip(res["metadatas"][0], res["distances"][0]):
    print("name:", meta["name"], "score:", 1-dist)
    print("text:", meta["text"])
    print("image:", meta["image_url"], "\n")

name: بنیامین بن موسی نهاوندی score: 0.8839473724365234
text: بنیامین بن موسی نهاوندی، حکیم، فیلسوف، عالم دینی، پزشک و عالم یهودی برجسته، در حدود قرن دوم شمسی در نهاوند واقع در استان همدان چشم به جهان گشود. او که در دوران قرون وسطی می زیست، نقشی اساسی در تکمیل و توسعه مبانی فرقه قرائیم ایفا نمود، جنبشی که توسط عنان ابن داوود پایه گذاری شده بود.  از جمله آثار ارزشمند او می توان به شروحی بر عهد عتیق و نوشته هایی به زبان عبری اشاره کرد.  نهاوندی با تاکید بر اهمیت تفکر آزاد و تحقیق، استدلال می کرد که تحقیق یک وظیفه است و اشتباهات ناشی از آن گناه محسوب نمی شود.
image:  

name: محمد بن طاهر بن بهرام سجستانی سیستانی score: 0.869530200958252
text: محمد بن طاهر بن بهرام سجستانی سیستانی، معروف به "منطقی"، در حدود سال ۲۹۱ هجری شمسی در سیستان چشم به جهان گشود. این فیلسوف و اندیشمند برجسته سده چهارم هجری، تحصیلات خود را در زمینه فلسفه و منطق در بغداد و نزد اساتیدی چون ابوبشر متی بن یونس و ابوزکریا یحی بن عدی گذراند. سجستانی، علاوه بر فعالیتهای علمی، از حمایت سیاسی حاکمان محلی و دیلمی نیز برخوردار ب

In [14]:
query_url = "https://upload.wikimedia.org/wikipedia/commons/thumb/e/e6/Birjandimanuscript.jpg/220px-Birjandimanuscript.jpg"
res = search_by_image(query_url, k=5)
for meta, dist in zip(res["metadatas"][0], res["distances"][0]):
    print("name:", meta["name"], "score:", 1-dist)
    print("text:", meta["text"])
    print("image:", meta["image_url"], "\n")

name: نظام الدین عبدالعلی بیرجندی score: 1.0
text: نظامالدین عبدالعلی بیرجندی، معروف به فاضل بیرجندی و محقق بیرجندی، دانشمند برجسته عصر صفوی بود. وی که در بیرجند واقع در خراسان جنوبی دیده به جهان گشود، در زمینههای ریاضیات و ستارهشناسی تبحر داشت. بیرجندی آثار متعددی از جمله "اسطرلاب"، "مختصر الهیئه" و "شرح التذکرة النصیریة فی الهیئة" را به رشته تحریر درآورد. او نزد اساتید بزرگی همچون غیاثالدین جمشید کاشانی و کمالالدین قنوی به تحصیل علوم مختلف پرداخت و سرانجام در سال ۹۳۴ هجری شمسی در بیرجند دار فانی را وداع گفت و در بجد به خاک سپرده شد.
image: https://upload.wikimedia.org/wikipedia/commons/thumb/e/e6/Birjandimanuscript.jpg/220px-Birjandimanuscript.jpg 

name: علی بن عباس مجوسی اهوازی score: 0.8311041593551636
text: علی بن عباس مجوسی اهوازی، پزشک و روانشناس برجسته سده چهارم هجری (دهم میلادی)، در شهر اهواز در ایران متولد شد. او که با القابی همچون "ابن المجوس" و "مجوسی اهوازی" نیز شناخته میشد، اثر ارزشمندی به نام "کتاب ملکی" از خود به یادگار گذاشته است. وی در حدود سال ۳۸۳ هجری شمسی درگذشت. 

# Generator